In [1]:
# pip install pyspark

In [2]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [1,039 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:1

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
Music_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Music_v1_00.tsv.gz"), sep="\t", header=True)
Music_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|     384427924|Whatever's for Us...|           Music|          5|            0|          0|   N|                Y|          Five Stars|Love this CD alon...| 2015-08-31|
|         US|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|     831769051|Same Trailer Diff...|           Music|          5|    

In [5]:
# Row count
Music_df.count()

4751577

In [6]:
from pyspark.sql.functions import to_date
# Review and cleaning DataFrame
review_id_df = Music_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3LI5TRP3YIDQL|   10140119|B00TXH4OLC|     384427924| 2015-08-31|
|R3LGC3EKEG84PX|   27664622|B00B6QXN6U|     831769051| 2015-08-31|
| R9PYL3OYH55QY|   45946560|B001GCZXW6|      14067376| 2015-08-31|
|R3PWBAWUS4NT0Q|   15146326|B000003EK6|     566295619| 2015-08-31|
|R15LYP3O51UU9E|   16794688|B00N1F0BKK|     210426072| 2015-08-31|
|R1AD7L0CC3DSRI|   32203364|B00V7KAO7Q|     360249504| 2015-08-31|
|R32FE8Y45QV434|    1194276|B000094Q4P|      20972283| 2015-08-31|
|R3NM4MZ4XWL43Q|   45813052|B00JMK0P1I|     378049560| 2015-08-31|
|R3H4FXX6Q7I37D|   12795687|B008OW1S3O|     141620702| 2015-08-31|
|R30L5PET7LFFDC|   36673840|B00VI2L3L4|     920557135| 2015-08-31|
|  REFRE1LEKLAF|   49453576|B0000041EV|     683128964| 2015-08-31|
|R3JTJ5EQN74E9H|    3285047|B00005YW4H|     268461622| 2015-08

In [7]:
products_df = Music_df.select(["product_id", "product_title"]).drop_duplicates()

In [8]:
reviews_df = Music_df.select(["review_id", "review_headline", "review_body"])
reviews_df.show(10)

+--------------+--------------------+--------------------+
|     review_id|     review_headline|         review_body|
+--------------+--------------------+--------------------+
|R3LI5TRP3YIDQL|          Five Stars|Love this CD alon...|
|R3LGC3EKEG84PX|A new fave in our...|This is the album...|
| R9PYL3OYH55QY|          Five Stars|  Excellent / thanks|
|R3PWBAWUS4NT0Q|         Three Stars|Nice variety of c...|
|R15LYP3O51UU9E|          Five Stars|Purchased as a gi...|
|R1AD7L0CC3DSRI|Definitely worth ...|Really enjoyed th...|
|R32FE8Y45QV434|          Five Stars|            Great CD|
|R3NM4MZ4XWL43Q|          Five Stars|       Excellent CD!|
|R3H4FXX6Q7I37D|          Four Stars|                nice|
|R30L5PET7LFFDC|        Outstanding!|Outstanding music...|
+--------------+--------------------+--------------------+
only showing top 10 rows



In [9]:
customers_df = Music_df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()


+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   16794688|             3|
|   49997672|             5|
|   16066470|             1|
|     636526|             1|
|   15963400|             8|
|   47840769|             4|
|    5884899|             7|
|   31220419|             2|
|   25410451|             8|
|   51052113|             4|
|   31562327|             2|
|   40710396|             5|
|   50988780|             4|
|   12374662|             4|
|   37007490|             1|
|    7328230|             1|
|   51154721|             1|
|   37153007|             1|
|   12318800|             2|
|   33775903|             1|
+-----------+--------------+
only showing top 20 rows



In [10]:
vine_df = Music_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3LI5TRP3YIDQL|          5|            0|          0|   N|
|R3LGC3EKEG84PX|          5|            0|          0|   N|
| R9PYL3OYH55QY|          5|            0|          1|   N|
|R3PWBAWUS4NT0Q|          3|            0|          0|   N|
|R15LYP3O51UU9E|          5|            0|          0|   N|
|R1AD7L0CC3DSRI|          5|            0|          0|   N|
|R32FE8Y45QV434|          5|            0|          0|   N|
|R3NM4MZ4XWL43Q|          5|            1|          2|   N|
|R3H4FXX6Q7I37D|          4|            0|          0|   N|
|R30L5PET7LFFDC|          5|            1|          1|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [11]:
mode = "append"
jdbc_url="jdbc:postgresql://mypostgressdb.cyn8iho1g7un.us-east-1.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"root", "password": "", "driver":"org.postgresql.Driver"}

In [12]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vines', mode=mode, properties=config)